#  Tutorial of Cryptography in Python

#### Guevara Noubir & Amirali Sanatinia
##### Network Security (CS 6740)


There are a number of crypto libraries in Python, (cryptography, pycrypto, m2crypto). This tutorial provides an overview of the cryptography library used in the CS 6740 Network Security course. You can download the library from [here](https://cryptography.io/) and follow the instructions. You should be able to install the library using the following command: 

```bash
pip install cryptography
```

Note that you need to have pip installed. To install pip, follow the instructions [here](https://pip.pypa.io/)

*note: Each code block has extra imports, so that blocks would be independent runnable code*

## Cyrptography.io
Cryptography components are divided into different submodules. Following is a list of these submodules (not exhaustive)

* Primitive Crypto Blocks (*cryptography.hazmat*)
 * Message Digest and Hashing algorithms (*cryptography.hazmat.primitives.hashes*)
 * Symmetric encryption algorithms (*cryptography.hazmat.primitives.ciphers*)
 * Asymmetric encryption algorithms (*cryptography.hazmat.primitives.asymmetric*)
* X.509 Ecosystem (*cryptography.x509*)
* Full high level crypto recipe (*cryptography.fernet*)


## Hashing Algorithms

As we discussed in class, the goal is to have a long message as input and produce an output which is much shorter called the hash or message digest. Furthermore, we want it to have properties such as pre-image, second preimage, and  collision resistance. *SHA* is a family of popular hash functions.


### Note on MD5

MD5 is hashing algorithm with block size of 512 bits, and digest size of 128 bit. It was designed by Ron Rivest in 1991. MD5 is considered insecure these days, and it is highly advised **not to be used in security systems**.

In [197]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64
digest = hashes.Hash(hashes.MD5(), backend=default_backend())
digest.update(b"Network")
digest.update(b"Security")
msg_digest = digest.finalize()

In [198]:
# Notice the output size of the digest
print "MD5", len(msg_digest), len(msg_digest) * 8, base64.b64encode(msg_digest)

MD5 16 128 CLAAes1RURgilKGMNbOdww==


In [199]:
digest = hashes.Hash(hashes.MD5(), backend=default_backend())
digest.update(b"NetworkSecurity")
msg_digest = digest.finalize()
print "MD5", len(msg_digest), len(msg_digest) * 8, base64.b64encode(msg_digest)

MD5 16 128 CLAAes1RURgilKGMNbOdww==


### SHA Family

Secure Hash Algorithm (SHA) family, is a series of hashing algorithms. Ranging from SHA-0 to SHA-3. SHA-0 should never be used, it's advised to move away from SHA-1 to SHA-2. Recent research indicates that SHA-1 is about to be broken. SHA-3 is the most recent version, published in 2015.

 * SHA-1: Digest size (160), Block size (512)
 * SHA-2: Digest size (224, 256, 384, or 512), Block size (512, 1024)
 * SHA-3: Digest size (224, 256, 384, 512), Block size (1600)

In [111]:
for _hash in [hashes.SHA1, hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512]:
    digest = hashes.Hash(_hash(), backend=default_backend())
    digest.update(b"Network")
    digest.update(b"Security")
    msg_digest = digest.finalize()
    # Notice the output size of the digest
    print _hash.name, len(msg_digest), len(msg_digest) * 8
    print base64.b64encode(msg_digest)

sha1 20 160
3Nfw+zQO2zp/8ZWVUdVm3hiCjmg=
sha224 28 224
WYm+UTU7ZMq4BAaX1+fwHRBtgt2KRE/ci+J1Zw==
sha256 32 256
Wr+fCs2Rfq5Q9nUtnl1pV8tNxrzYIhzR+8TPdyWMLjg=
sha384 48 384
I121lIgEBkunsrznkTk8wvV1rYJT27CxN5w3vND36C9E+bUnfp/carfxDdgi6b0n
sha512 64 512
NRJrp7SwnPeOgak1bffPaKtUfoboqWsmNtp/ybIyhgEtggqeuGvW+uZ5TBcTbQNDnlihqxKnHGtnFuDgN4iSEw==


In [112]:
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"Network")
digest.update(b"Security")
msg_digest = digest.finalize()

In [113]:
print base64.b64encode(msg_digest)

Wr+fCs2Rfq5Q9nUtnl1pV8tNxrzYIhzR+8TPdyWMLjg=


In [115]:
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"NetworkSecurity")
msg_digest = digest.finalize()

In [116]:
print base64.b64encode(msg_digest)

Wr+fCs2Rfq5Q9nUtnl1pV8tNxrzYIhzR+8TPdyWMLjg=


### Hash-based message authentication code (HMAC)

HMAC is used for message authentications combined with a secret key. It provides integrity check and authentication.

<img src="include/SHAhmac.png">

image source: wikipedia

In [119]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
import os
for _hash in [hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512]:
    h = hmac.HMAC(os.urandom(16), _hash(), backend=default_backend())
    h.update("Network Security")
    msg_digest = h.finalize()
    print _hash.name, len(msg_digest), len(msg_digest) * 8, base64.b64encode(msg_digest)

sha224 28 224 /3hfD/Av0s6CsxgL2DzCDrX+aA70GgVmkn1T9w==
sha256 32 256 2XLF0i0+eV/j/dSUCiyRozUByKcE/mdanzHC/cpAUAk=
sha384 48 384 3h5ZC8IwF6IC2M9BcolFhM+JDj4JSxmbFk0Aw+cxeC264jjddRZSMx345REKP4SE
sha512 64 512 V7zEa/pnUXrPazLizq/wWZpzSAikeNUGl1iWdXcvmkp+08ZzTFDRZrX7khs+TzH2V2/67KmyWA9dQmZ7aP6ouw==


## Symmetric Encryption

In the following we look at the symmetric encryption algorithms. In symmetric crpto, we use the same key for encryption and decryption. Therefore, the two parties needs to establish a secret key between them. It's up to 1000 times faster than asymmetric encryption.


### Advanced Encryption Algorithm (AES)

AES is based on Rijndael encryption algorithm, designed by Joan Daemen and Vincent Rijmen. It was one of the algorithms submitted to U.S. National Institute of Standards and Technology (NIST) to replace DES and 3DES. It was published in 1998 and accepted and standardized in 2001.

 * AES supports key sizes of 128/192/256 bits
 * Block size: 128 bit
 * It's iterative rather than Feistel cipher
 * Treats data in 4 groups of 4 bytes
 * Operates on an entire block in every round
 * Resistant against known attacks
 * Speed and code compactness on many CPUs
 * Rijndael block and key size vary between 128, 192, 256
 * However, in AES block size in 128
 * Number of rounds a function of key size
  * 128 bits     10 rounds
  * 192 bits     12 rounds
  * 256 bits     14 rounds

 * Today most implementations use the CPU support (Intel AES-NI)

### Block cipher mode of operation

To encrypt messages of arbitrary size with block ciphers, we use the following algorithms, called the modes of operation. They define how to encrypt each block of the plaintext to produce the corresponding cipher text block. Some of these are complemetly insecure (ECB) and should not be used.

 * Electronic Codebook (ECB)
 * Cipher Block Chaining (CBC)
 * Cipher Feedback (CFB)
 * Output Feedback (OFB)
 * Counter (CTR)
 * Galois Counter Mode (GCM)
 
 
### Electronic Codebook (ECB)

<img src="include/ECB_enc.png">
<img src="include/ECB_dec.png">



### Cipher Block Chaining (CBC)

<img src="include/CBC_enc.png">
<img src="include/CBC_dec.png">



### Counter (CTR)

<img src="include/CTR_enc.png">
<img src="include/CTR_dec.png">

image source: wikipedia

The following images are encrypted with ECB. Note that you can see the pattern in the data. Therefore, ECB is not secure or recommended to be used.

<img src="include/tux.png">
<img src="include/ECB1.png">
<img src="include/ECB2.png">

In [210]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
key = os.urandom(16) # in bytes, 128 bits
iv = os.urandom(16)

In [211]:
# ECB Mode, we only need a key
### *** DO NOT ECB. IT IS INSECURE *** ###

cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
cipher_text = encryptor.update("Network Security1234567890123456") + encryptor.finalize()
print base64.b64encode(cipher_text)

ioEIeuMVaCO5op9SLtnRpSJTBrRxlxIdy/4LQqCx1g4=


In [212]:
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
ct1 = encryptor.update("Network Security1234567890123457")
encryptor.finalize()
print base64.b64encode(ct1)

ioEIeuMVaCO5op9SLtnRpSnfMUWNT+kxyxN+b1s/F24=


In [213]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

'Network Security1234567890123456'

In [214]:
# CBC Mode, we also need an IV
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
cipher_text = encryptor.update("Network Security") + encryptor.finalize()

In [215]:
base64.b64encode(cipher_text)

'jC3zZ6lrFLQXqAnJkjyJRA=='

In [216]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

'Network Security'

In [217]:
# GCM Mode, we also need an IV
cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
encryptor.authenticate_additional_data(b"Only authenticate")
cipher_text = encryptor.update("Network Security") + encryptor.finalize()
tag = encryptor.tag

In [218]:
base64.b64encode(cipher_text)

'O5JJenYBN6vF7ZzSyXvM8w=='

In [219]:
decryptor = Cipher(algorithms.AES(key), modes.GCM(iv,tag), backend=default_backend()).decryptor()
decryptor.authenticate_additional_data(b"Only authenticate")
decryptor.update(cipher_text) 
decryptor.finalize()

''

In [57]:
# CTR Mode, we don't need padding in CTR mode. In transforms a block cipher into a stream cipher
# we only need to introduce the nonce
cipher = Cipher(algorithms.AES(key), modes.CTR(os.urandom(16)), backend=default_backend())
encryptor = cipher.encryptor()
# len("Network Security CS 6740") = 25, but no padding is needed
cipher_text = encryptor.update("Network Security CS(6740)") + encryptor.finalize()

In [59]:
base64.b64encode(cipher_text)

'j9GNBIMaCYn+xPaKYLaK8dydv2P+MfZIqQ=='

In [60]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

'Network Security CS(6740)'

## Asymmetric Encryption

Asymmetric encryption mechanism use two different keys for encryption and decryption. Therefore, the two parties do not need to share a secret key between them.


### RSA

RSA, is an asymmetric encryption algorithm by Ron Rivest, Adi Shamir, and Leonard Adleman. It was published in 1977. It's security is based on the hardness of factorization problem. However, now it has its own problem, called the RSA problem. RSA is slow, and is not used for encryptin large data, but it's mostly used to encrypt the symmetric key that is used for encryption.


 * p, q, two big prime numbers (private, chosen)
 * n = pq, f(n) = (p-1)(q-1)   (public, calculated)
 * e, with gcd(f(n), e) = 1,  1 < e < f(n)	(public, chosen)
 * d = e-1 mod f(n)	(private, calculated)
 * $E(M) = M^e \mod n$
 * $D(M) = M^d \mod n$
 * $D(E(M)) = M^{ed} \mod n = M$
 
 


In [184]:
# Generate a 2048 bit private key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend())
# to get the public key
public_key = private_key.public_key()

<img src="include/RSA_OAEP.png">

image souce: wikipedia

In [192]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"The SECRET KEY"
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA1(),
        label=None))
base64.b64encode(ciphertext)

'e2BT1CJzJFmExEDmxZeVAHKmZmsliEQuR+uRmEbHs+iW7IQCjSF6xSZAAsd5nGzZqiLYqp4MWomuvQ7kvORW3meGLzCLE3Z9XqYLbzgjjpqKyl9Gv8mqcB0O1HsMNqnCXfBr3oTGrkKsqxRz8IqScLQtkBneSIW9EPrzaWxPMKcKAnIUDmOvOJ0RptAFMmw9huSFLnp4Bs520q9tcwxhHSCQbMdrubycBh697v/2TT+1o1AB2XgMKmM+O23RGNqrRznGZjPDKQloOrp/LAJPLVhHE3xPBAeEfCASJByNNyETTEEi3BaEnXPeAV7VXWKPGOIxMPhR0oCjNFwb4OuQaQ=='

In [193]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA1(),
        label=None))
plaintext

'The SECRET KEY'

In [194]:
plaintext == message

True

### OpenSSL - RSA

To generate keys, use the following instructions:

```bash
 openssl genrsa -out private_key.pem 2048
 openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
 openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der
 ```
 

In [220]:
%%bash
openssl genrsa -out private_key.pem 2048
openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der

Generating RSA private key, 2048 bit long modulus
.....................+++
.......................................................................................................+++
e is 65537 (0x10001)
writing RSA key


In [196]:
# import key from a file. E.g., previously generated by OpenSSL
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("private_key.pem", "rb") as key_file:
     private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend())
public_key = private_key.public_key()